# 1. Loading Imports

In [ ]:
import pandas as pd
import numpy as np


import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder



# 2. Reading Data

In [ ]:
test = pd.read_csv("../input/englishpremiership/test/test/2021-2022.csv")

_0 = pd.read_csv("../input/englishpremiership/train/train/2020-2021.csv")
_1 = pd.read_csv("../input/englishpremiership/train/train/2019-2020.csv")
_2 = pd.read_csv("../input/englishpremiership/train/train/2018-2019.csv")
_3 = pd.read_csv("../input/englishpremiership/train/train/2017-2018.csv")
_4 = pd.read_csv("../input/englishpremiership/train/train/2016-2017.csv")
_5 = pd.read_csv("../input/englishpremiership/train/train/2015-2016.csv")

_6 = pd.read_csv("../input/englishpremiership/train/train/2014-2015.csv")
train = pd.concat([_0,_1,_2,_3,_4,_5,_6])

In [ ]:
FEATURES = ['Time', 'HomeTeam', 'AwayTeam', 'FTR','MaxH' ,'MaxD' ,'MaxA' ,'AvgH' ,'AvgD' ,'AvgA']

## a) Train Data

In [ ]:
test = test[FEATURES]
train.head()

## b) Test Data

In [ ]:
test = test[FEATURES]
test.head()

# 2. Data Preprocessing

In [ ]:
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

## a) Categorical Variables

In [ ]:
# Get list of categorical variables
s = (train[FEATURES].dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
# Make copy to avoid changing original data 
label_train = train[FEATURES].copy()
label_test = test[FEATURES].copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_train[object_cols] = ordinal_encoder.fit_transform(label_train[object_cols])
label_test[object_cols] = ordinal_encoder.fit_transform(label_test[object_cols])

## b) Missing Values

i. Here we calculate the number of missing values.

In [ ]:
# Shape of training data (num_rows, num_columns)
print(label_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (label_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
print(label_test.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (label_test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
# Imputation
my_imputer = SimpleImputer()
imputed_train = pd.DataFrame(my_imputer.fit_transform(label_train))
imputed_test = pd.DataFrame(my_imputer.transform(label_test))

# Imputation removed column names; put them back
imputed_train.columns = label_train.columns
imputed_test.columns = label_test.columns

In [ ]:
print(imputed_test.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (imputed_test.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
print(imputed_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (imputed_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
train_X_full = imputed_train.drop("FTR", axis=1)
test_X_full = imputed_test.drop("FTR", axis=1)

# train_X_full = imputed_train
# test_X_full = imputed_test

categorical_cols = [cname for cname in train_X_full.columns if train_X_full[cname].nunique() < 10 and 
                        train_X_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in train_X_full.columns if train_X_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols

# train_X = train_X_full[my_cols].copy()
# test_X = test_X_full[my_cols].copy()

train_X = train_X_full.copy()
test_X = test_X_full.copy()

train_y = label_train.FTR
test_y = label_test.FTR

In [ ]:
X = pd.concat([train_X, test_X])
y = pd.concat([train_y, test_y])

In [ ]:
print(X.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
print(y.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (y.isnull().sum())
print(missing_val_count_by_column)

In [ ]:
y

## 3. Pipeline

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


# 4. Models

In [ ]:
train_y = train_y.fillna(0)

## a) Base Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

basic_model = RandomForestRegressor(random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', basic_model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(train_X, train_y)

# Preprocessing of validation data, get predictions
base_preds = my_pipeline.predict(test_X)

# Evaluate the model
base_score = mean_absolute_error(test_y, base_preds)
print('MAE:', base_score)
# print(base_preds)
# print(test_y)

## b) Random Forest Regressor Model

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
rf_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', rf_model)
                             ])

# Preprocessing of training data, fit model 
rf_pipeline.fit(train_X, train_y)

# Preprocessing of validation data, get predictions
rf_preds = rf_pipeline.predict(test_X)

# Evaluate the model
rf_score = mean_absolute_error(test_y, rf_preds)
print('MAE:', rf_score)
print(rf_preds)

## c) Random Forest Classifier Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rfc_model = RandomForestClassifier(n_estimators=100)

rfc_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', rfc_model)
                             ])

# Preprocessing of training data, fit model 
rfc_pipeline.fit(train_X, train_y)

# Preprocessing of validation data, get predictions
rfc_preds = rfc_pipeline.predict(test_X)

# Evaluate the model
rfc_score = mean_absolute_error(test_y, rfc_preds)
print('MAE:', rfc_score)
# print(rfc_preds)
# print(test_y)

## d) XGBoost Model

In [ ]:
xgb_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)

# Bundle preprocessing and modeling code in a pipeline
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_model)
                             ])

# Preprocessing of training data, fit model 
xgb_pipeline.fit(train_X, train_y)

# Preprocessing of validation data, get predictions
xgb_preds = xgb_pipeline.predict(test_X)

# Evaluate the model
xgb_score = mean_absolute_error(test_y, xgb_preds)
print('MAE:', xgb_score)
print(xgb_preds)

# 5. Model Validation

In [ ]:
y = y.fillna(0)

In [ ]:
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print()

print("Average MAE score (across experiments):")
print(scores.mean())

In [ ]:
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(rf_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print()

print("Average MAE score (across experiments):")
print(scores.mean())


In [ ]:
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(rfc_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print()

print("Average MAE score (across experiments):")
print(scores.mean())

print()

cv_scores = cross_val_score(rfc_pipeline, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

In [ ]:
# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(xgb_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print()

print("Average MAE score (across experiments):")
print(scores.mean())

# 6. Improvements (Feature Engineering)

## a) Mutual Information

In [ ]:
def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

discrete_features = X.dtypes == int
mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores # show a few features with their MI scores

In [ ]:
def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")


plt.figure(dpi=100, figsize=(8, 5))
plot_mi_scores(mi_scores)

In [ ]:
sns.relplot(x="FTAG", y="FTR", data = pd.concat([train,test]));

In [ ]:
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=6)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

X.head()

In [ ]:
sns.relplot(
    y="HomeTeam", x="MaxD", hue="Cluster", data=X, height=6,
)

In [ ]:
# X["MedHouseVal"] = df["MedHouseVal"]
sns.catplot(x="Time", y="Cluster", data=X, kind="boxen", height=6)

In [ ]:
X_scaled = (X[numerical_cols] - X[numerical_cols].mean(axis=0)) / X[numerical_cols].std(axis=0)
X_scaled.head()

In [ ]:
# Shape of training data (num_rows, num_columns)
print(X_scaled.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_scaled.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
# Create principal components
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

X_pca.head()

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,  # transpose the matrix of loadings
    columns=component_names,  # so the columns are the principal components
    index=X_scaled.columns,  # and the rows are the original features
)
loadings

In [ ]:
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)


def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

In [ ]:
# Look at explained variance
plot_variance(pca)

In [ ]:
mi_scores = make_mi_scores(X_pca, y.fillna(0), discrete_features=False)
mi_scores

In [ ]:
# Since there is no preprocessing, we don't need a pipeline (used anyway as best practice!)

train_X_pca, test_X_pca, train_y, test_y = train_test_split(X_pca, y, train_size=0.8, test_size=0.2, random_state=0)

rfc_pca_model = RandomForestClassifier(n_estimators=100)

# rfc_pca_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('model', rfc_pca_model)
#                              ])

# Preprocessing of training data, fit model 
rfc_pca_model.fit(train_X_pca, train_y)

# Preprocessing of validation data, get predictions
rfc_pca_preds = rfc_pca_model.predict(test_X_pca)

# Evaluate the model
rfc_pca_score = mean_absolute_error(test_y, rfc_pca_preds)
print('MAE:', rfc_pca_score)
print()
print('Predictions\n', rfc_pca_preds)
print()
print('Test set\n', test_y.values)

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(rfc_pca_model, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print()

print("Average MAE score (across experiments):")
print(scores.mean())
print()

cv_scores = cross_val_score(rfc_pca_model, X, y, 
                            cv=5,
                            scoring='accuracy')

print("Cross-validation accuracy: %f" % cv_scores.mean())

1. Position they finished last year.
2. Division in which they were playing last year.
3. Parse Date and Time.
4. Total goals scored last year.
5. Total goals conceded last year.
6. Number of cleansheets.
7. Number of top 5 goal scorers in team.